<h1> <center> Bank Marketing Analysis </center> </h1>

<h3> </center> Objective: </center> </h3> 

The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe a term deposit (variable y).

**Repo Link:** https://github.com/UBC-MDS/dsci_522_group_8.git

In [ ]:
## Install Necessary Packages
#!pip install ucimlrepo

In [ ]:
## Import
#Necessary Packages
import altair as alt
import altair_viewer
import pandas as pd
import numpy as np
import statistics


#Data 
from ucimlrepo import fetch_ucirepo 

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# fetch dataset 
bank_marketing = fetch_ucirepo(id=222) 
  
# data (as pd dataframes) 
X = bank_marketing.data.features 
y = bank_marketing.data.targets 
  
# metadata 
#display(bank_marketing.metadata) 
  
# variable information 
display(bank_marketing.variables) 

<h3> <center> Preparing the Dataset: </center> </h3>

In [ ]:
df = X
#Drop NA values
df.dropna()

#Imputing Missing using kNN Imputer, Simple Imputer
knn_imp = KNNImputer(n_neighbors = 4)
simple_imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
df["poutcome"] = simple_imp.fit_transform(df[["poutcome"]]).ravel()
df["contact"] = simple_imp.fit_transform(df[["contact"]]).ravel()
df["pdays"] = knn_imp.fit_transform(df[["pdays"]]).ravel()

#Drop some values (~288 of 45,000) that are hard to impute 
df = df.dropna()

#Add target column
df["target"] = y

<h3> <center> Exploratory Data Analysis </center> </h3>

In [ ]:
#Summary Statistics
display(pd.DataFrame(df.describe()).T)
colnames = df.columns.tolist()

In [ ]:
#Distribution
numeric = ["age", "balance", "day_of_week", "duration", "campaign", "pdays", "previous"]

charts = []

# Create density plots for each numeric column
for column in numeric:
    chart = alt.Chart(df).transform_density(
        column,
        groupby=["target"],  # Assuming "target" is a column in your DataFrame
        as_=[column, "density"]
    ).mark_area(opacity=0.5).encode(
        x=alt.X(f"{column}:Q").scale(zero = False),
        y=alt.Y("density:Q").stack(False).scale(zero = False),
        color="target"
    )
    charts.append(chart)

In [ ]:
grid = alt.concat(*charts, columns=2)
grid.show()